In [ ]:
from pathlib import Path

repo_root = Path.cwd()               
schema_base_dir = (repo_root/"data"/"retail_db").as_posix()
print(schema_base_dir)


In [ ]:
import json
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType, LongType,TimestampType,IntegerType
from pyspark.sql import functions as F,Window
from pathlib import Path
import os
import sys
repo_root = Path.cwd()  
# java_home = r"C:\Users\pedro.lourenco\AppData\Local\Programs\Eclipse Adoptium\jdk-17.0.17.10-hotspot"
# os.environ["JAVA_HOME"] = java_home
# os.environ["PATH"] = str(Path(java_home) / "bin") + ";" + os.environ["PATH"]

# # ✅ força SPARK_HOME para o pyspark do venv
# pyspark_dir = Path(pyspark.__file__).resolve().parent  # ...\site-packages\pyspark
# os.environ["SPARK_HOME"] = str(pyspark_dir)

# print("SPARK_HOME:", os.environ["SPARK_HOME"])
# print("spark-submit.cmd:", (pyspark_dir / "bin" / "spark-submit.cmd").exists())
# os.environ["PYSPARK_PYTHON"] = sys.executable
# os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
spark = (
    SparkSession.builder
    .appName("TesteLocal")
    .master("local[*]")
    # # ✅ desativa NativeIO no Hadoop (evita o access0)
    # .config("spark.hadoop.io.nativeio.native.disable", "true")
    # # ✅ também ajuda em Windows
    # .config("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem")
    # .config("spark.hadoop.fs.AbstractFileSystem.file.impl", "org.apache.hadoop.fs.local.LocalFs")
    # # (opcional) evita certas otimizações locais
    # .config("spark.hadoop.dfs.client.read.shortcircuit", "false")
    # .config("spark.hadoop.dfs.client.use.legacy.blockreader.local", "false")
    # .config("spark.pyspark.python", sys.executable)
    # .config("spark.pyspark.driver.python", sys.executable)
    # .config("spark.local.dir", r"C:\temp\spark_local")
    .getOrCreate()
)

print("Spark version:", spark.version)

print(spark.sparkContext.applicationId)
print(spark.sparkContext.uiWebUrl)

Spark version: 3.5.2
local-1767353222937
http://c21ccc96-d470-4a63-ab62-9cdb0f48db98.internal.cloudapp.net:4040


In [ ]:
schema = StructType([
    StructField("stock_id",    StringType(), True),
    StructField("trans_date",  StringType(), True),
    StructField("open_price",  FloatType(),  True),
    StructField("low_price",   FloatType(),  True),
    StructField("high_price",  FloatType(),  True),
    StructField("close_price", FloatType(),  True),
    StructField("volume",      LongType(),   True)
])

df = spark.read.csv(
    _files_path(r"C:\Users\pedro.lourenco\GCP\PySpark\data\nyse_all\nyse_data","*.txt.gz"),
    schema=schema,
    header=True,
    sep=","
)

In [ ]:
# df.write.mode("overwrite").parquet(r"C:\Users\pedro.lourenco\GCP\PySpark\data\nyse_all\nyse_data\nyse_data_parquet")

In [ ]:
df.cache()

DataFrame[stock_id: string, trans_date: string, open_price: float, low_price: float, high_price: float, close_price: float, volume: bigint]

In [ ]:
dict(df.dtypes)

{'stock_id': 'string',
 'trans_date': 'string',
 'open_price': 'float',
 'low_price': 'float',
 'high_price': 'float',
 'close_price': 'float',
 'volume': 'bigint'}

In [ ]:
df.printSchema()

root
 |-- stock_id: string (nullable = true)
 |-- trans_date: string (nullable = true)
 |-- open_price: float (nullable = true)
 |-- low_price: float (nullable = true)
 |-- high_price: float (nullable = true)
 |-- close_price: float (nullable = true)
 |-- volume: long (nullable = true)



In [ ]:
df.count()

9384718

In [ ]:
count_filter=(
df
#  df.filter(F.col("stock_id" ) == "ABRN")
 .groupBy("stock_id")
 .agg(F.count("*").alias("num_records"))
)

In [ ]:
count_filter.show()

+--------+-----------+
|stock_id|num_records|
+--------+-----------+
|     BOX|       1603|
|     CCK|       5230|
|    CNXC|        405|
|     CRS|       5230|
|     EFR|       3431|
|     FMY|       3039|
|     GIS|       5230|
|     IHD|       1495|
|       K|       5230|
|     LEN|       5230|
|     MHF|       5230|
|     PKE|       5230|
|     TLI|       4738|
|     TNP|       3881|
|     TYG|       3365|
|    SPGI|        190|
|     AAT|       1569|
|     AIV|       5230|
|     AVX|       5230|
|     AVY|       5230|
+--------+-----------+
only showing top 20 rows



In [ ]:
w = Window.partitionBy("stock_id").orderBy(F.desc("trans_date"))

count_filter = (
    df
    # .filter(F.col("stock_id") == "ABRN")
    .select(
        "stock_id",
        "trans_date",
        "close_price"
    )
    .withColumn("num_records",F.row_number().over(w))
)

In [ ]:
count_filter.show()

+--------+----------+-----------+-----------+
|stock_id|trans_date|close_price|num_records|
+--------+----------+-----------+-----------+
|      AA|  20170117|      32.64|          1|
|      AA|  20170116|      33.01|          2|
|      AA|  20170113|      33.01|          3|
|      AA|  20170112|      33.04|          4|
|      AA|  20170111|      31.97|          5|
|      AA|  20170110|      30.98|          6|
|      AA|  20170109|      29.48|          7|
|      AA|  20170106|      30.68|          8|
|      AA|  20170105|      30.65|          9|
|      AA|  20170104|      30.26|         10|
|      AA|  20170103|      28.83|         11|
|      AA|  20170102|      28.08|         12|
|      AA|  20161230|      28.08|         13|
|      AA|  20161229|      28.89|         14|
|      AA|  20161228|      29.43|         15|
|      AA|  20161227|      29.65|         16|
|      AA|  20161226|      29.71|         17|
|      AA|  20161223|      29.71|         18|
|      AA|  20161222|      29.75| 

In [ ]:
w = Window.partitionBy("stock_id").orderBy(F.desc("trans_date"))

count_filter = (
    df
    .filter(F.col("stock_id") == "ABRN")
    .select(
        "stock_id",
        "trans_date",
        "close_price"
    )
    .withColumn("num_records", F.row_number().over(w))
)

In [ ]:
count_filter.explain(True)


# Json Load FIles

In [11]:
import warnings


type_mapping = {
    "integer": IntegerType(),
    "string": StringType(),
    "timestamp": TimestampType(),
    "float": FloatType()
}

def _paths(folder_path,kind: str,pattern:str = "*"):
    
    kind = kind.lower()
    
    if kind not in {"file", "folder"}:
        raise ValueError("Nenhum file/folder path atribuido")
    
    base_path = Path(folder_path)
    
    if kind == "file":
        path = [path.as_posix() for path in base_path.glob(pattern) if path.is_file()]

    else:
        path = [path.as_posix() for path in base_path.glob(pattern) if path.is_dir()]
    
    if not path:
        warnings.warn(f"[WARN] Nenhum {kind} encontrado em {base_path} com pattern='{pattern}'")
    return path

def _load_schema_json(schema_paths) -> dict:

    schema_path = schema_paths[0] if isinstance(schema_paths, (list, tuple)) else schema_paths
    
    with open(schema_path, "r", encoding="utf-8") as f:
        return json.load(f)

def _build_schema(table_name: str, schema_json: dict) -> StructType:
    if table_name not in schema_json:
        raise KeyError(f"Tabela {table_name} não encontrada no JSON de schema")
    fields = sorted(schema_json[table_name], key= lambda col: col["column_position"], reverse = False)
    
    return StructType(
       [ StructField(
            field["column_name"]
            ,type_mapping.get(field["data_type"].lower(),StringType())
            ,True
        )
        for field in fields]
    )

schema_base_dir = (repo_root/"data"/"retail_db").as_posix()
output_dir_parq = f"{schema_base_dir}_parquet"
schema_paths = _paths(schema_base_dir,"file", "schemas*")
schema_json = _load_schema_json(schema_paths)
ds_list = _paths(schema_base_dir,"folder")

for ds in ds_list:
    ds = Path(ds).name
    print(f"Processing {ds} data")
    
    schema_table = _build_schema(ds,schema_json)
    files=_paths(f"{schema_base_dir}/{ds}","file", "part-*")
    if not files:
        continue
    print(files)

    df = (
        spark.read
        .schema(schema_table)
        .option("header", "false")
        .option("sep", ",")
        .option("mode", "PERMISSIVE")
        .csv(files)
    )
    
    output_dir = (Path(output_dir_parq)/ds).as_posix()
    print(output_dir)
    df.show(5)
    # (
    #     df.write
    #     .mode("overwrite")      # ou "append"
    #     .parquet(output_dir)
    # )
    



/tmp/ipykernel_4366/3148205491.py:27: UserWarning: [WARN] Nenhum file encontrado em /workspaces/Spark-CodeSpace/data/retail_db com pattern='schemas*'
  warnings.warn(f"[WARN] Nenhum {kind} encontrado em {base_path} com pattern='{pattern}'")


IndexError: list index out of range

In [ ]:
de_group = (
    df
    .groupBy("order_status")
    .agg(F.count("*").alias("order_count"))
    .orderBy(F.col("order_count").desc())
)

In [ ]:
de_group.explain(True)

== Parsed Logical Plan ==
'Sort ['order_count DESC NULLS LAST], true
+- Aggregate [order_status#762], [order_status#762, count(1) AS order_count#772L]
   +- Relation [order_id#759,order_date#760,order_customer_id#761,order_status#762] csv

== Analyzed Logical Plan ==
order_status: string, order_count: bigint
Sort [order_count#772L DESC NULLS LAST], true
+- Aggregate [order_status#762], [order_status#762, count(1) AS order_count#772L]
   +- Relation [order_id#759,order_date#760,order_customer_id#761,order_status#762] csv

== Optimized Logical Plan ==
Sort [order_count#772L DESC NULLS LAST], true
+- Aggregate [order_status#762], [order_status#762, count(1) AS order_count#772L]
   +- Project [order_status#762]
      +- Relation [order_id#759,order_date#760,order_customer_id#761,order_status#762] csv

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [order_count#772L DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(order_count#772L DESC NULLS LAST, 200), ENSURE_REQ

In [ ]:
df.show(truncate= False)

+--------+---------------------+---------------------+---------------+
|order_id|order_date           |order_customer_id    |order_status   |
+--------+---------------------+---------------------+---------------+
|1       |2013-07-25 00:00:00.0|+11599-01-01 00:00:00|CLOSED         |
|2       |2013-07-25 00:00:00.0|NULL                 |PENDING_PAYMENT|
|3       |2013-07-25 00:00:00.0|+12111-01-01 00:00:00|COMPLETE       |
|4       |2013-07-25 00:00:00.0|8827-01-01 00:00:00  |CLOSED         |
|5       |2013-07-25 00:00:00.0|+11318-01-01 00:00:00|COMPLETE       |
|6       |2013-07-25 00:00:00.0|7130-01-01 00:00:00  |COMPLETE       |
|7       |2013-07-25 00:00:00.0|4530-01-01 00:00:00  |COMPLETE       |
|8       |2013-07-25 00:00:00.0|2911-01-01 00:00:00  |PROCESSING     |
|9       |2013-07-25 00:00:00.0|5657-01-01 00:00:00  |PENDING_PAYMENT|
|10      |2013-07-25 00:00:00.0|5648-01-01 00:00:00  |PENDING_PAYMENT|
|11      |2013-07-25 00:00:00.0|NULL                 |PAYMENT_REVIEW |
|12   